This is the test code for checking the speed of neighbor_list ase vs matscipy vs torch_geometric vs pymatgen

In [1]:
from ase.io import read
from ase.neighborlist import primitive_neighbor_list # ase
from matscipy.neighbours import neighbour_list # matscipy
# torch_geometric
import torch
from torch_geometric.data import Data
from torch_geometric.transforms import RadiusGraph 

# pymatgen
from pymatgen.core import Structure
from pymatgen.analysis.local_env import CutOffDictNN
from pymatgen.analysis.graphs import StructureGraph

import sys
import numpy as np
from mace.data.neighborhood import get_neighborhood
from sevenn.train.dataload import unlabeled_atoms_to_graph
import matplotlib.pyplot as plt

/home/hexagonrose/miniconda3/envs/mp/lib/python3.9/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.lo

In [3]:
atoms = read('POSCAR_Li28La12Zr8O48')
pos = atoms.get_positions()
cell = np.array(atoms.get_cell())
cutoff = 5.0
pbc = atoms.get_pbc()

## Time check
---
### ASE

In [4]:
%%timeit -n 10
# ase.neighborlist
edge_src, edge_dst, edge_vec, shifts = primitive_neighbor_list(
        'ijDS', pbc, cell, pos, cutoff, self_interaction=False
    )

128 ms ± 2.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Matscipy

In [5]:
%%timeit
# matscipy.neighbours
edge_src, edge_dst, edge_vec, shifts = neighbour_list(
        quantities="ijDS",
        pbc=pbc,
        cell=cell,
        positions=pos,
        cutoff=5.0,
        # self_interaction=True,  # we want edges from atom to itself in different periodic images
        # use_scaled_positions=False,  # positions are not scaled positions
    )

4.42 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Identical Graph?
---

### ASE

In [6]:
edge_src_ase, edge_dst_ase, edge_vec_ase, shifts_ase = primitive_neighbor_list(
        'ijDS', pbc, cell, pos, cutoff, self_interaction=False
    )

In [15]:
print(edge_src_ase, len(edge_src_ase))
print(edge_dst_ase, len(edge_dst_ase))
print(edge_vec_ase, len(edge_vec_ase))
print(shifts_ase, len(shifts_ase))

[ 0  0  0 ... 95 95 95] 4288
[60 62 63 ...  5  8 21] 4288
[[-0.4581798  -4.04339437  0.36458977]
 [-2.57464063  0.4581798  -2.81083673]
 [-2.57464063 -0.4581798   3.54001627]
 ...
 [-1.26486532  2.22320973 -4.08024523]
 [ 0.55612672  2.22320973 -0.90481873]
 [ 1.8889394   0.15410094 -0.02657832]] 4288
[[ 0  0 -1]
 [ 0  0 -1]
 [ 0  0 -1]
 ...
 [ 0  0  1]
 [ 0  0  1]
 [ 0  0  1]] 4288


### Matscipy

In [11]:
edge_src_mat, edge_dst_mat, edge_vec_mat, shifts_mat = neighbour_list(
        quantities="ijDS",
        pbc=pbc,
        cell=cell,
        positions=pos,
        cutoff=5.0,
        # self_interaction=True,  # we want edges from atom to itself in different periodic images
        # use_scaled_positions=False,  # positions are not scaled positions
    )

In [16]:
print(edge_src_mat, len(edge_src_mat))
print(edge_dst_mat, len(edge_dst_mat))
print(edge_vec_mat, len(edge_vec_mat))
print(shifts_mat, len(shifts_mat))

[ 0  0  0 ... 95 95 95] 4288
[ 6  9 10 ... 49 66 89] 4288
[[-4.21951352  2.39852148  0.        ]
 [-2.39852148 -4.21951352  0.        ]
 [-2.39852148 -2.39852148  3.1754265 ]
 ...
 [ 2.4964684   0.57833684  1.906018  ]
 [ 4.31059092 -1.26880967  0.24680685]
 [ 0.95834441  0.95834441  4.54121554]] 4288
[[ 0  0 -1]
 [ 0  0 -1]
 [ 0  0 -1]
 ...
 [ 0  1  1]
 [ 0  1  1]
 [ 0  1  1]] 4288


### Torch_geometric

In [34]:
pos_tensor = torch.tensor(pos, dtype=torch.float)
data = Data(pos=pos_tensor)
data = RadiusGraph(5.0)(data)

ImportError: 'radius_graph' requires 'torch-cluster'

In [33]:
structure = Structure.from_file("POSCAR_Li28La12Zr8O48")
cutoff = 5.0

# Use a cutoff-based nearest neighbors strategy
nn_strategy = CutOffDictNN(cut_off_dict={"Li": cutoff, "La": cutoff, "Zr": cutoff, "O": cutoff})
# structure_graph = StructureGraph.with_local_env_strategy(structure, nn_strategy)

# Extract edges (source, destination) and edge vectors
# edges = structure_graph.graph.edges(data=True)
# edge_src, edge_dst, edge_vec = [], [], []

# for u, v, d in edges:
#     edge_src.append(u)
#     edge_dst.append(v)
#     edge_vec.append(d["to_jimage"])  # Fractional lattice vector

# print("Pymatgen edges:", len(edge_src))

ValueError: not enough values to unpack (expected 2, got 1)

In [20]:
import numpy as np

# Assuming edge_src_ase, edge_dst_ase, edge_vec_ase, shifts_ase
# are from ASE and similarly for matscipy

# Combine source and destination into edge pairs
edges_ase = np.array(list(zip(edge_src_ase, edge_dst_ase)))
edges_matscipy = np.array(list(zip(edge_src_mat, edge_dst_mat)))

# Sort the edges for consistent ordering
sorted_edges_ase = edges_ase[np.lexsort((edges_ase[:, 1], edges_ase[:, 0]))]
sorted_edges_matscipy = edges_matscipy[np.lexsort((edges_matscipy[:, 1], edges_matscipy[:, 0]))]

# Compare edges
edges_equal = np.array_equal(sorted_edges_ase, sorted_edges_matscipy)

# Compare edge vectors
sorted_vec_ase = np.array(edge_vec_ase)[np.argsort(edge_src_ase)]
sorted_vec_matscipy = np.array(edge_vec_mat)[np.argsort(edge_src_mat)]
edge_vec_equal = np.allclose(sorted_vec_ase, sorted_vec_matscipy, atol=1e-6)

# Compare shifts
sorted_shifts_ase = np.array(shifts_ase)[np.argsort(edge_src_ase)]
sorted_shifts_matscipy = np.array(shifts_mat)[np.argsort(edge_src_mat)]
shifts_equal = np.array_equal(sorted_shifts_ase, sorted_shifts_matscipy)

# Final result
if edges_equal and edge_vec_equal and shifts_equal:
    print("The graphs from ASE and matscipy are identical!")
else:
    print("The graphs from ASE and matscipy are different.")


The graphs from ASE and matscipy are different.


In [23]:
sorted_edges_ase

array([[ 0,  4],
       [ 0,  5],
       [ 0,  6],
       ...,
       [95, 89],
       [95, 91],
       [95, 92]])

In [24]:
sorted_edges_matscipy

array([[ 0,  4],
       [ 0,  5],
       [ 0,  6],
       ...,
       [95, 89],
       [95, 91],
       [95, 92]], dtype=int32)